In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
from rdkit.Chem import MolStandardize

# syntax edit 1 - no longer openforcefield.topology  
from openff.toolkit import ForceField, Molecule, Topology
from openff.toolkit.topology import Molecule
# syntax edit 2
from openff.toolkit.typing.engines.smirnoff import ForceField as OFFForceField
from openmmforcefields.generators import SystemGenerator
from simtk import openmm, unit
from simtk.openmm import app
from simtk.openmm.app import PDBFile
# syntax edit 3 
from openmm.app.modeller import Modeller 
from simtk.openmm.app import NoCutoff, HBonds



In [2]:
from pdbfixer import PDBFixer
import pdb4amber
import parmed
import MDAnalysis

In [3]:


#                                -----   LIGAND PREP  -----



In [4]:
# syntax edit 4 - -h replaced by -p 7 (to take into account the protonatuon states when H is added)

#adding hydrogens to ligand (ligand.pdb is delinker output)
!obabel ligand.pdb -ipdb -opdb -O ligand_h.pdb -p 7

1 molecule converted


In [5]:
#convert to sdf
!obabel ligand_h.pdb -ipdb -osdf -O ligand_h.sdf


1 molecule converted


In [6]:
#create pdb object
pdbfile = PDBFile('./ligand_h.pdb')
#create molecule object
mol = Molecule.from_file('./ligand_h.sdf')


# HELPFUL TIP 1 : can view topology at this stage to check by executing: "pdbfile.topology"

/Users/fififoufa/mambaforge/envs/openff/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (ATOM     35  H   UNK     0      15.798  -5.475  19.565  1.00  0.00           H  , ATOM     34  H   UNK     0      16.517  -6.551  20.738  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/Users/fififoufa/mambaforge/envs/openff/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (ATOM     36  H   UNK     0      14.971  -6.944  20.025  1.00  0.00           H  , ATOM     35  H   UNK     0      15.798  -5.475  19.565  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/Users/fififoufa/mambaforge/envs/openff/lib/python3.9/site-packages/openmm/app/internal/pdbstr

In [7]:
# Create the Open Force Field Topology from an OpenMM Topology object.
##
omm_topology = pdbfile.topology

# syntax edit 5 - ligand obj added , allow undefined added, .to_topology()applied to ligand

ligand = Molecule.from_file('./ligand_h.sdf', allow_undefined_stereo=True)
off_topology = ligand.to_topology()
openmm_topology = off_topology.to_openmm()


# Load the OpenFF "Parsley" force field.
forcefield = OFFForceField('openff-1.0.0.offxml') 

# Parametrize the topology and create an OpenMM System.
system = forcefield.create_openmm_system(off_topology)

/Users/fififoufa/mambaforge/envs/openff/lib/python3.9/site-packages/openff/interchange/components/interchange.py:339: UserWarning: Automatically up-converting BondHandler from version 0.3 to 0.4. Consider manually upgrading this BondHandler (or <Bonds> section in an OFFXML file) to 0.4 or newer. For more details, see https://openforcefield.github.io/standards/standards/smirnoff/#bonds.
  warnings.warn(


In [8]:
# # HELPFUL TIP 2 : downloading the protein used in the tutorial like so ensures its accessibility from the github without issues 

!wget https://raw.githubusercontent.com/bieniekmateusz/Protein-ligand-openmm-workflow/main/mpro.pdb

--2023-03-08 15:56:20--  https://raw.githubusercontent.com/bieniekmateusz/Protein-ligand-openmm-workflow/main/mpro.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244671 (239K) [text/plain]
Saving to: ‘mpro.pdb.5’

mpro.pdb.5          100%[===================>] 238.94K  --.-KB/s    in 0.04s   

2023-03-08 15:56:20 (5.71 MB/s) - ‘mpro.pdb.5’ saved [244671/244671]



In [9]:
omm_topology = pdbfile.getTopology()
positions = pdbfile.getPositions()

# Convert OpenMM System into a ParmEd Structure.
ligand_structure = parmed.openmm.load_topology(off_topology.to_openmm(),
                                                   system,
                                                   xyz=mol.conformers[0])



/Users/fififoufa/mambaforge/envs/openff/lib/python3.9/site-packages/parmed/structure.py:1701: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  coords = np.array(value, dtype=np.float64, copy=False, subok=True)


In [10]:


#                                -----   PROTEIN PREP  -----




In [11]:
#fix PDB
fixer = PDBFixer('./mpro.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
PDBFile.writeFile(fixer.topology, fixer.positions, open('mpro_nosolvent.pdb', 'w'))





In [12]:
#more PDB fixing + protonation
pdb4amber.run(arg_pdbout='mproh.pdb', arg_pdbin='mpro_nosolvent.pdb', arg_reduce=True, arg_logfile='pdb4amber.log')
!tail -n 20 pdb4amber.log


Summary of pdb4amber for: mpro_nosolvent.pdb
REDUCE returned non-zero exit status: See reduce_info.log for more details

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None


In [13]:
#make the system generator, load forcefields
forcefield_kwargs = { 'constraints' : app.HBonds, 'rigidWater' : True, 'removeCMMotion' : False, 'hydrogenMass' : 4*unit.amu }

#system generator used to create system objects
system_gen = SystemGenerator(forcefields=['amber99sbildn.xml', 'tip3pfb.xml'], 
                             small_molecule_forcefield="openff-1.3.0.offxml", 
                             forcefield_kwargs=forcefield_kwargs,
                            molecules=[mol,])

In [14]:
#make the modeller
pdbfile = PDBFile('mproh.pdb')
top = pdbfile.getTopology()
pos = pdbfile.getPositions()
modeller = app.Modeller(top, pos)

In [15]:
#now add water 
modeller.addSolvent(system_gen.forcefield, model="tip3p", padding=1, ionicStrength=0*unit.molar, neutralize=False)
top = modeller.topology
pos = modeller.positions

#write a pdb file of solvated protein
with open("mpro_h_solvent.pdb", 'w') as outfile:
    app.PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

In [16]:
#parameterize the protein and create a system 
system = system_gen.create_system(topology=modeller.topology)

In [17]:
#convert the protein system into a ParmEd structure
protein_structure = parmed.openmm.load_topology(top,
                                           system,
                                           xyz=pos)

In [18]:
#write structure to pdb file
PDBFile.writeFile(protein_structure.topology, 
                  protein_structure.positions, 
                  open('protein_structure.pdb', 'w'))
pdbfile = PDBFile('protein_structure.pdb')

In [19]:


#                                -----   Combining the protein and ligand  -----





In [20]:
#adding the structures from protein.ipynb and ligand.ipynb
complex_structure = protein_structure + ligand_structure
#writing new structure to a .pdb file
complex_structure.write_pdb('./complex_system.pdb')

In [21]:


#                                -----   Simulating the protein-ligand comple  -----






In [22]:
#convert the structure to an openmm system
complex_system = system_gen.create_system(topology=complex_structure.topology)

/Users/fififoufa/mambaforge/envs/openff/lib/python3.9/site-packages/openff/interchange/components/interchange.py:339: UserWarning: Automatically up-converting BondHandler from version 0.3 to 0.4. Consider manually upgrading this BondHandler (or <Bonds> section in an OFFXML file) to 0.4 or newer. For more details, see https://openforcefield.github.io/standards/standards/smirnoff/#bonds.
  warnings.warn(


In [23]:
#propagate the System with Langevin dynamics.
time_step = 1*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator_min = openmm.LangevinIntegrator(temperature, friction, time_step)

#set up an openmm simulation
simulation = openmm.app.Simulation(complex_structure.topology, complex_system, integrator_min)


#set the initial positions
positions = complex_structure.positions

simulation.context.setPositions(positions)


In [24]:
simulation.minimizeEnergy()
simulation.saveState('./minimized.state')

In [25]:
# HELPFUL TIP 3 : PRINT OUT MINIMISED COORDINATES AS A PDB - COMPARE WITH ORIGINAL PDB

In [26]:
#get state of minimised simulation
state = simulation.context.getState(getPositions=True)
#get positions of minimised simulation
positions = state.getPositions()
#write minimised structure to .pdb
PDBFile.writeFile(complex_structure.topology, positions, open("minimised_complex.pdb", "w"))
pdbfile_min = PDBFile('./minimised_complex.pdb')

In [28]:
# syntax edit 6 - use pdfile_min for .topology istead of any other object

minimised_structure = parmed.openmm.load_topology(pdbfile_min.topology, complex_system,xyz=positions)

In [29]:
#propagate the System with Langevin dynamics.
time_step = 1*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator_prod = openmm.LangevinIntegrator(temperature, friction, time_step)

#length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 1  # number of steps per written trajectory frame
data_freq = 1  # number of steps per written simulation statistics

#set up an OpenMM simulation using minimised structure
simulation = openmm.app.Simulation(minimised_structure.topology, complex_system, integrator_prod)

#set the initial positions.
positions = pdbfile_min.getPositions()
simulation.context.setPositions(positions)


#randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

#configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter('trajectory_prod.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter('data_prod.csv', data_freq, step=True,
                                                   potentialEnergy=True, temperature=True,
                                                   density=True)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)



In [ ]:
import time

print("Starting simulation")
start = time.process_time()

#run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end-start))
print("Done!")

Starting simulation
